In [2]:
import os
from PIL import Image, ImageOps
import numpy as np

In [32]:
from license_plate_recognition.segmentation.service import SegmentationService

segmentation_service = SegmentationService()

In [61]:
def segment_directory(directory: str):
    result = []
    for file in os.listdir(directory):
        print(file)
        file_path = os.path.join(directory, file)
        pillow_image = Image.open(file_path)
        image = np.array(pillow_image)

        characters = segmentation_service.segment(image)

        for character in characters:
            character_image = Image.fromarray(character)
            result.append(character_image)
    
    return result

In [51]:
# run segmentation on the train plates
results_directory = os.path.join(os.getcwd(), "data")

test_directory = os.path.join(os.getcwd(), "..", "..", "samples", "train")
segmented_characters = segment_directory(test_directory)

for index, character in enumerate(segmented_characters):
    character.save(os.path.join(results_directory, f"{index + 1}.png"))


9d49300bf30e4574adb6a868dc9427a4.jpg
polen52_jpg.rf.8a2a57ab10abf625343c0bb7735578ce.jpg
c79e1f7c1ea04ed9b1dd96f99075e097.jpg
3369f832f45847668f1419200277efe3.jpg
kasonk_jpg.rf.9a0a733392b9feedd7fdf0fa7256419f.jpg
88b844b183544bf4a41b5ab34c22a8b6.jpg
40cd8fc86ee2493e970b42a36b6e647d.jpg
dbcc7e6b7b474062844f07c68d00f6f8.jpg
b5276ab20f3e4276ae87137cd460f1e0.jpg
c7dd86bc6c2f42dbbce1457593e06bff.jpg
fe03f120b4ef4345a53e206e35b80620.jpg
3d9d85c031fe443683da62ed62d25d38.jpg
d07173ad8be2469daaf0b82bb28a85d2.jpg
f840562cec0f4e1682982f0022f5ec0c.jpg
cdbb3ad928f34809866773530cf772ca.jpg
pmkpasnk_jpg.rf.bc988de36424df916824ac6aeac0d3f9.jpg
a3ee430dab6445c08270d0ad23d505c6.jpg
d862d05b9edb4b2eac833f6af718f659.jpg
4d0a98d2d6f44d198198b40aee96380c.jpg
15559233878d469481b113b6938e11e7.jpg
0ad1fdaec5184722af46e5ed54eb8a4f.jpg
5189f2d49a2e46bd84f1427df0a8b624.jpg
00dcbd4d83b94d1d80cf6df7af8b9093.jpg
ljajkl_jpg.rf.d816983ad8f43c0b36001157d78051f5.jpg
657f6d3054a64d88b929045a0000a064.jpg
4f69ca8c667948c0

In [49]:
# create directories to sort the segmented characters
sorted_directory = os.path.join(os.getcwd(), "data", "sorted")

if not os.path.exists(sorted_directory):
    os.mkdir(sorted_directory)
    
for i in range(10):
    directory = os.path.join(sorted_directory, str(i))
    if not os.path.exists(directory):
        os.mkdir(directory)

for i in range(65, 91):
    directory = os.path.join(sorted_directory, chr(i))
    if not os.path.exists(directory):
        os.mkdir(directory)


In [62]:
# check how many samples in every category after manual sorting
sorted_directory = os.path.join(os.getcwd(), "data", "sorted")

for category in sorted(os.listdir(sorted_directory)):
    category_directory = os.path.join(sorted_directory, category)
    if not os.path.isdir(category_directory):
        continue
    category_items = len(os.listdir(category_directory))
    print(f"{category}: {category_items}")

0: 46
1: 51
2: 64
3: 45
4: 51
5: 49
6: 52
7: 63
8: 54
9: 58
A: 25
B: 13
C: 27
D: 3
E: 17
F: 11
G: 12
H: 12
I: 12
J: 19
K: 36
L: 27
M: 19
N: 7
O: 16
P: 7
Q: 0
R: 12
S: 127
T: 29
U: 9
V: 11
W: 21
X: 6
Y: 8
Z: 13


In [63]:
# adjust image size
EXPECTED_WIDTH = 27
EXPECTED_HEIGHT = 40

def adjust_image_size(image):
    img = Image.fromarray(image)
    # resize keeping the aspect ratio
    img = ImageOps.contain(img, (EXPECTED_WIDTH, EXPECTED_HEIGHT), Image.Resampling.NEAREST)
    
    # add padding if the image is smaller than the expected size
    array = np.array(img)
    if array.shape[0] < EXPECTED_HEIGHT:
        new_array = np.zeros((EXPECTED_HEIGHT, array.shape[1]), dtype=np.uint8)
        start = (EXPECTED_HEIGHT - array.shape[0]) // 2
        new_array[start:(array.shape[0] + start), :] = array
        array = new_array
    
    if array.shape[1] < EXPECTED_WIDTH:
        new_array = np.zeros((array.shape[0], EXPECTED_WIDTH), dtype=np.uint8)
        start = (EXPECTED_WIDTH - array.shape[1]) // 2
        new_array[:, start:(array.shape[1] + start)] = array
        array = new_array
        
    return array

In [64]:
# Prepare training data
classes_directory = os.path.join(os.getcwd(), "data", "sorted")
classes = sorted(filter(lambda x: os.path.isdir(os.path.join(classes_directory, x)), os.listdir(classes_directory)))

x_train = []
y_train = []
for index, c in enumerate(classes):
    class_directory = os.path.join(classes_directory, c)
    for file in os.listdir(class_directory):
        if not file.endswith(".png"):
            continue
        
        file_path = os.path.join(class_directory, file)
        pillow_image = Image.open(file_path)
        image = np.array(pillow_image)
        image = adjust_image_size(image)
        # scale input data to [0,1]
        image = image / 255.0
        x_train.append(image)
        y_train.append(index)
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [67]:
import keras
from keras import layers, models

# Build the model (fully connected neural network)
model = models.Sequential([
    keras.Input(shape=(40, 27)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(len(classes), activation='softmax')
], name="RecognizePlateCharacter")

# Other possible solutions
# https://www.tensorflow.org/tutorials/images/classification
# CNN
# model = models.Sequential([
#     layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
#     layers.Conv2D(16, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(num_classes)
# ])

# Prepare for training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, epochs=10)


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3653 - loss: 2.6941  
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8040 - loss: 0.7802 
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8785 - loss: 0.5248 
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9373 - loss: 0.2754 
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9453 - loss: 0.2072 
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9697 - loss: 0.1637 
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9680 - loss: 0.1486 
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9754 - loss: 0.1054 
Epoch 9/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9813 - loss: 0.0948 
Epoch 10/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9879 - loss: 0.0719 


In [60]:
# Save the model
model.save("./recognition_model.keras")